# **LAMS Local Air Pollution Monitoring System**
Particulate pollution estimation from an input image.

**Loading the Dataset**
Image data is collected from the surroundings of the College
Data labeling is done using the date of the image captured and the corresponding AQI value is obtained from the AP Pollution board website
https://pcb.ap.gov.in/UI/AmbientAirQualityMonitoringData.aspx

In [ ]:
!git clone https://github.com/BhavyaM-23/sky

In [ ]:
!pip install ktrain

In [ ]:
%matplotlib inline
import os
import ktrain
from ktrain import vision as vis

In [ ]:
import re
pattern=r'([^/]+)_\d+.jpg$'
p=re.compile(pattern)
r=p.search('33_20211120131613.jpg')
print(r.group(1))

In [ ]:
DATADIR='/content/sky'
dir=os.listdir(DATADIR)
act=[]
dir.remove('.git')
dir.remove('.ipynb_checkpoints')
for file in dir:
  print(file)
  r=p.search(file)
  act.append(int(r.group(1)))
print(act)

In [ ]:
(train_data,test_data,preproc)=vis.images_from_fname(DATADIR,pattern=pattern,is_regression=True,random_state=42)

In [ ]:
vis.print_image_regression_models()

**RESNET-50 pretrained model is used**

In [ ]:
model=vis.image_regression_model('pretrained_resnet50',train_data=train_data,val_data=test_data)

In [ ]:
learner=ktrain.get_learner(model=model,train_data=train_data,val_data=test_data,batch_size=10)

In [ ]:
#learning rate=1e-4

In [ ]:
learner.fit_onecycle(1e-4,15)

In [ ]:
#learner.freeze(15)
#learner.fit_onecycle(1e-4,3)

In [ ]:
learner.model.summary()

In [ ]:
predictor=ktrain.get_predictor(learner.model,preproc)

In [ ]:
test_data.filenames

**Sample Prediction**

In [ ]:
def show_prediction(fname):
  fname=DATADIR+'/'+fname
  pred=round(predictor.predict_filename(fname)[0])
  actual=int(p.search(fname).group(1))
  vis.show_image(fname)
  print("Predicted AQI: %s | Actual AQI: %s"%(pred,actual))

In [ ]:
show_prediction('33_2007002478.jpg')

**Saving the model**

In [ ]:
from tensorflow import keras

kf="m1.h5"
keras.models.save_model(learner.model,kf)

In [ ]:
import tensorflow as tf
conv=tf.lite.TFLiteConverter.from_keras_model(learner.model)
tf=conv.convert()
open("aqi.tflite","wb").write(tf)

In [ ]:
predictor.predict_filename('/content/sky/37_20210725175039.jpg')[0]

In [ ]:
val=[]
for f in dir:
  val.append(int(predictor.predict_filename(DATADIR+'/'+f)[0]))
print(val)

In [ ]:
print(act)

**Error rate**

In [ ]:
eff=[]
for i in range(len(act)):
  x=abs(act[i]-val[i])/act[i]
  eff.append(x)
print(sum(eff)/len(eff))